In [1]:
import h5py
import numpy as np

f = h5py.File("data/train_digitStruct.mat", "r")
bbox_refs = f["digitStruct"]["bbox"]
names = f["digitStruct"]["name"]

def read_attr(attr):
    """Read a bbox attribute (left/top/width/height/label)."""
    # If it's a single reference
    if len(attr) == 1:
        return attr[0][0]
    # If multiple references
    return np.array([a[0] for a in attr]).astype(float)

def get_boxes(i):
    # Dereference the bbox for image i
    bbox = f[bbox_refs[i][0]]

    def get_field(name):
        val = bbox[name]
        # val can be scalar or array of references
        if len(val) == 1:
            return val[0][0]
        else:
            return np.array([v[0] for v in val]).astype(float)

    return {
        "left":   get_field("left"),
        "top":    get_field("top"),
        "width":  get_field("width"),
        "height": get_field("height"),
        "label":  get_field("label")
    }

def read_string(ref):
    return "".join([chr(c[0]) for c in f[ref]])

def get_name(i):
    name_ref = names[i][0]
    return read_string(name_ref)

# names[1]

# Example: first image
filename = get_name(993)
box_info = get_boxes(993)
print(filename, box_info)





994.png {'left': np.float64(157.0), 'top': np.float64(29.0), 'width': np.float64(102.0), 'height': np.float64(175.0), 'label': np.float64(4.0)}
